In [1]:
import numpy as np
from numpy import linalg as LA
from numpy.linalg import inv
from scipy.linalg import rq
import math

In [2]:
# X = np.array([
#     [4.74, 4.74, 0],
#     [7.11, 7.11, 0],
#     [7.11, 11.85, 0],
#     [11.85, 11.85, 4.74],
#     [11.85, 7.11, 4.74],
#     [11.85, 4.74, 7.11]
# ]
# )
X = np.array([
    [2,2,0],
    [3,5,0],
    [5,5,2],
    [5,2,3],
    [5,6,3],
    [5,4,1]
]
)


In [3]:
# x = np.array([
#     [356, 836],
#     [450, 815],
#     [581, 886],
#     [682, 700],
#     [556, 804],
#     [528, 532]
# ]
# )
x = np.array([
    [356, 836],
    [581, 886],
    [682, 700],
    [528, 532],
    [789, 694],
    [580, 702]
]
)

In [4]:
def normalizing_params(pts):
    centroid=np.mean(pts,axis=0)
    avg_distance=np.mean(LA.norm(pts-centroid, axis=1))
    return centroid,avg_distance


In [5]:
def normalizing_matrix(pts):
    c,d=normalizing_params(pts)
    n=pts.shape[1]
    if n==2:
        d=d/math.sqrt(2)
        mat=np.array([[1/d,0,-c[0]/d],[0,1/d,-c[1]/d],[0,0,1]])
    else:
        d=d/math.sqrt(3)
        mat=np.array([[1/d,0,0,-c[0]/d],[0,1/d,0,-c[1]/d],[0,0,1/d,-c[2]/d],[0,0,0,1]])
    return mat

In [6]:
def homo_coord(pts):
    rows=pts.shape[0]
    pts=np.append(pts, np.ones((rows, 1)), axis=1)
    return pts

In [7]:
def Fiducial_matrix(X_pts,x_pts):
    rows=X_pts.shape[0]
    X_pts_homo=homo_coord(X_pts)
    x_pts_homo=homo_coord(x_pts)

    T=normalizing_matrix(x_pts)
    U=normalizing_matrix(X_pts)
    # print(T)
    # print(U)
    x_pts_homo=(np.dot(T,x_pts_homo.T)).T
    X_pts_homo=(np.dot(U,X_pts_homo.T)).T

    
    zerocol=np.zeros((rows,4))
    multx=(-X_pts_homo.T*x_pts_homo[:,0]).T
    multy=(-X_pts_homo.T*x_pts_homo[:,1]).T

    P=np.append(X_pts_homo,zerocol,axis=1)
    P=np.append(P,multx,axis=1)
    P=np.append(P,zerocol,axis=1)
    P=np.append(P,X_pts_homo,axis=1)
    P=np.append(P,multy,axis=1)
    
    P=np.reshape(P,(2*rows,12))
    return P

In [8]:
# Fiducial_matrix(X,x)

In [9]:
def denormalize(X_pts,x_pts,P):
    T=normalizing_matrix(x_pts)
    U=normalizing_matrix(X_pts)
    P=np.dot(inv(T),P)
    P=np.dot(P,U)
    return P

In [10]:
def P_estimate(X_pts,x_pts):
    P=Fiducial_matrix(X_pts,x_pts)
    u, s, vh = np.linalg.svd(P)
    least_val_vector=vh[-1]
    P=np.reshape(least_val_vector,(3,4))
    P=denormalize(X_pts,x_pts,P)
    return P

In [21]:
def P_decomposition(P):
    KR=P[:,0:3]
    K, R = rq(KR)
    KRX=P[:,-1]
    X_0 = np.dot(-np.linalg.inv(KR),KRX)
    return K,R,X_0
    # print(r)
    # print(q)

In [22]:
# print(normalizing_matrix(x))#T
# print(normalizing_matrix(X))#U

In [23]:
P=P_estimate(X,x)
print(P)

[[ 2.56592115e+01  4.60951586e+01  1.07832658e+01  6.18561828e+01]
 [-1.07943168e+01  3.05750883e+01 -4.42653005e+01  4.42695680e+02]
 [ 3.14083369e-02  9.12628459e-03 -2.50249878e-02  4.95793551e-01]]


In [15]:
X_test = np.array([
    [0, 0, 0],
    [11.85, 9.48, 0],
    [4.74, 14.22, 0],
    [11.85, 14.22, 7.11],
    [9.48, 9.48, 0],
    [11.85, 9.48, 2.37]
]
)

In [16]:
x_actual = np.array([
    [124, 887],
    [542, 750],
    [632, 980],
    [789, 694],
    [530, 798],
    [580, 705]
])

In [17]:
def test_pred(P, X_test):
  rows=X_test.shape[0]
  X_test = np.append(X_test, np.ones((rows,1)), axis=1)
  # print(X_test)
  x_test = np.dot(P,X_test.T)
  x_test = x_test/x_test[2]
  x_test = x_test[0:2].T
  return x_test

In [18]:
# x_predict=test_pred(P,X_test)
x_predict=test_pred(P,X)

In [19]:
def rms_error(x_predict, x_actual):
    return np.sqrt(np.mean((x_predict-x_actual)**2))

In [20]:
# rms_error(x_predict,x_actual)
rms_error(x,x_predict)

0.4486850397607956

In [56]:
def params1(P):
    K,R,X0=P_decomposition(P)
    alpha=K[0][0]
    theta=np.arctan(-alpha/K[0][1])
    theta_degree=theta*180/np.pi
    beta=K[1][1]*np.sin(theta)
    x0=K[0][2]
    y0=K[1][2]
    skew=K[0][1]
    t=-np.dot(R,X0)
    return alpha,beta,theta_degree,x0,y0,skew,R,X0,t
    
    

In [57]:
alpha,beta,theta_degree,x0,y0,skew,R,X0,t=params1(P)

In [58]:
X0

array([-6.61574648, -0.32386539, 11.39054574])

In [59]:
t

array([-4.56185248, -2.80590272, 12.03885104])

In [60]:
P.shape

(3, 4)

In [71]:
def radial_dist(P,X_test):
    rows=X_test.shape[0]
    X_test = np.append(X_test, np.ones((rows,1)), axis=1)
    # print(X_test)
    x_test = np.dot(P,X_test.T)
    # x_test = x_test/x_test[2]
    x_test = x_test.T
    return x_test

In [72]:
radial_dist(P,X)

array([[205.36492307, 482.25722261,   0.57686279],
       [369.30961049, 563.18817086,   0.63564998],
       [442.19456504, 453.06893626,   0.64841668],
       [314.6923549 , 317.07837072,   0.59601284],
       [499.07298944, 439.3787241 ,   0.63251798],
       [385.31614064, 466.75914843,   0.66431539]])